In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
from pandas import set_option
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import statsmodels as stm
import patsy
import pymc

In [2]:
#loading the allcars dataset from Yun Solutions
set_option('display.max_rows', 2)
data = pd.read_csv('allcars.csv', nrows = 240000, index_col=0)
#data = data.drop(['Unnamed: 0'], axis = 1)
data

,timeStamp,tripID,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos,deviceID
0,2017-10-31 18:45:59.000000,0,ffb0fd700220fff43fffe74808e73e03f33400ef33fbe1...,2.2,14.48,NaN,NaN,44.313725,NaN,NaN,0.0,0.0,903.0,4.0,NaN,15.686275,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16065,2017-09-21 20:10:29.000000,2,0ca800f007980009c50108c3ff07c20008c20108c30009...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4


In [3]:
#take two vehicles
dfv1 = pd.DataFrame(); dfv2 = pd.DataFrame();
dfv1 = data.loc[data.deviceID == 1]; dfv2 = data.loc[data.deviceID == 2]

In [4]:
set_option('display.max_rows', 10)
dfv1.describe()

,tripID,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos,deviceID
count,117275.000000,117275.000000,117275.000000,0.0,0.0,117275.000000,0.0,0.0,117275.000000,117275.000000,117275.000000,117275.000000,0.0,117275.000000,117275.0
mean,50.327896,8.114516,12.437785,NaN,NaN,46.855066,NaN,NaN,0.003199,0.406219,852.304570,15.390501,NaN,17.561893,1.0
std,22.836783,11.257505,4.295417,NaN,NaN,29.376612,NaN,NaN,0.103067,12.948273,901.577115,21.706871,NaN,7.183419,0.0
min,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,1.0
25%,29.000000,0.000000,12.880000,NaN,NaN,23.921569,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,15.686275,1.0
50%,54.000000,0.000000,14.000000,NaN,NaN,52.156863,NaN,NaN,0.000000,0.000000,902.000000,0.000000,NaN,20.000000,1.0
75%,71.000000,14.900000,14.260000,NaN,NaN,70.196078,NaN,NaN,0.000000,0.000000,1516.000000,28.000000,NaN,20.392157,1.0
max,87.000000,52.900000,48.914000,NaN,NaN,99.607843,NaN,NaN,13.304381,593.990000,12228.500000,191.000000,NaN,74.901961,1.0


In [5]:
dfv2.describe()

,tripID,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos,deviceID
count,106659.000000,106659.000000,106659.000000,106460.000000,106460.0,106659.000000,106460.000000,106460.000000,106659.000000,106659.000000,106659.000000,106659.000000,106460.0,106659.000000,106659.0
mean,63.579239,18.188948,0.009114,64.143575,0.0,35.413494,31.122901,96.442175,0.000664,0.025123,1182.769860,33.042819,0.0,0.001324,2.0
std,38.780889,18.726188,0.665543,29.107386,0.0,22.531762,15.797552,47.344598,0.048472,1.834499,762.717486,34.009781,0.0,0.096652,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.0
25%,25.000000,0.000000,0.000000,60.000000,0.0,22.352941,24.000000,97.000000,0.000000,0.000000,800.750000,0.000000,0.0,0.000000,2.0
50%,60.000000,13.600000,0.000000,80.000000,0.0,38.823529,34.000000,99.000000,0.000000,0.000000,1165.750000,25.000000,0.0,0.000000,2.0
75%,99.000000,27.700000,0.000000,81.000000,0.0,48.235294,43.000000,110.000000,0.000000,0.000000,1732.500000,50.000000,0.0,0.000000,2.0
max,126.000000,82.100000,48.607000,84.000000,0.0,94.901961,58.000000,221.000000,3.540058,133.980000,5508.500000,157.000000,0.0,7.058824,2.0


In [6]:
#Deleting less important variables
dfv1 = dfv1.drop(['cTemp','dtc','iat','imap','tAdv','deviceID'], axis = 1)
dfv2 = dfv2.drop(['dtc','tAdv','deviceID'], axis = 1)

In [7]:
#Selecting the largest trip of each vehicle
